# Загрузка данных из PostgreSQL

В этом ноутбуке мы рассмотрим загрузку данных из PostgreSQL с использованием GPB библиотеки.

## Подготовка окружения

In [ ]:
import sys
sys.path.append('../../')

import pandas as pd
import numpy as np
from core.data.schema import DataSchema, ColumnSchema, DataType
from core.data.data_loaders import PostgresDataLoader

## Создание схемы данных

In [ ]:
# Создание схемы данных
schema = DataSchema([
    ColumnSchema(
        name='id',
        data_type=DataType.INTEGER,
        required=True,
        constraints={
            'unique': True
        }
    ),
    ColumnSchema(
        name='name',
        data_type=DataType.STRING,
        required=True
    ),
    ColumnSchema(
        name='age',
        data_type=DataType.INTEGER,
        required=True,
        constraints={
            'min': 0,
            'max': 150
        }
    ),
    ColumnSchema(
        name='created_at',
        data_type=DataType.DATETIME,
        required=True
    )
])

## Создание загрузчика данных

In [ ]:
# Создание загрузчика данных
loader = PostgresDataLoader(
    connection_params={
        'host': 'localhost',
        'port': 5432,
        'database': 'mydb',
        'user': 'myuser',
        'password': 'mypassword',
        'use_ssl': True
    },
    schema=schema
)

## Загрузка данных из таблицы

In [ ]:
# Загрузка всех данных из таблицы
try:
    df = loader.load_data("SELECT * FROM users")
    print("Данные успешно загружены из таблицы:")
    print(df)
except Exception as e:
    print(f"Ошибка при загрузке данных: {e}")

## Загрузка данных с параметрами

In [ ]:
# Загрузка данных с параметрами
query = """
    SELECT * FROM users
    WHERE age > %(min_age)s
    AND created_at > %(min_date)s
"""

params = {
    'min_age': 18,
    'min_date': '2023-01-01'
}

try:
    df = loader.load_data(query, params=params)
    print("Данные успешно загружены с параметрами:")
    print(df)
except Exception as e:
    print(f"Ошибка при загрузке данных: {e}")

## Загрузка данных с JOIN

In [ ]:
# Загрузка данных с JOIN
query = """
    SELECT u.*, o.order_id, o.amount
    FROM users u
    LEFT JOIN orders o ON u.id = o.user_id
    WHERE u.age > %(min_age)s
"""

try:
    df = loader.load_data(query, params={'min_age': 18})
    print("Данные успешно загружены с JOIN:")
    print(df)
except Exception as e:
    print(f"Ошибка при загрузке данных: {e}")

## Обработка ошибок

In [ ]:
# Пример обработки ошибок при загрузке
try:
    # Попытка подключения с неверными параметрами
    loader = PostgresDataLoader(
        connection_params={
            'host': 'wrong_host',
            'port': 5432,
            'database': 'mydb',
            'user': 'myuser',
            'password': 'mypassword',
            'use_ssl': True
        },
        schema=schema
    )
    df = loader.load_data("SELECT * FROM users")
except Exception as e:
    print(f"Ошибка подключения: {e}")